In [1]:
from dotenv import load_dotenv
import os
import sys
load_dotenv()
sys.path.append(os.getenv('BASE_DIR'))
from Efficientunet.liver_abd_efficientunet import get_efficientunet_b0_parallel, get_efficientunet_b0_shared_decoder, get_efficientunet_b0
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset
from shared_functions.data import ImageMaskDataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



In [2]:
from shared_functions.metrics import dice_coef, fpr, hausdorff_distance, tpr, dice_loss_multi, dice_loss
from shared_functions.data import load_data_loaders
n_splits = 3

n_epochs = 50
batch_size = 4

torch.cuda.empty_cache()

dataloader_dir = os.getenv("DATALOADER_MULTI_DIR")
train_loaders, test_loaders = load_data_loaders(os.path.join(dataloader_dir, 'train_loaders.pth'), os.path.join(dataloader_dir, 'test_loaders.pth'))
alpha = 0.5

C:\Users\Jeff\Desktop\liver_ultrasound\clean\shared_functions\data.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_loaders = torch.load(train_path)
C:\Users\Jeff\D

In [3]:
decoder_type = 'parallel'
performance_file = os.getenv('BASE_DIR') + f'/Efficientunet/results/{decoder_type}/performance.txt'
model_dir = os.getenv('BASE_DIR') + f'/Efficientunet/results/{decoder_type}/models/'
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

for fold in range(n_splits):
    train_loader = train_loaders[fold]
    test_loader = test_loaders[fold]

    model = get_efficientunet_b0_parallel(out_channels=1, concat_input=False, pretrained=False).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=5e-4, weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)

    criterion = dice_loss()

    # Training
    for epoch in range(n_epochs):
        model.train()
        running_loss = 0.0
        for images, _, liver_masks, abd_masks in train_loader:
            images, liver_masks, abd_masks = images.to(device), liver_masks.to(device), abd_masks.to(device)

            optimizer.zero_grad()
            liver_outputs, abd_outputs = model(images)
            liver_outputs, abd_outputs = torch.clamp(liver_outputs, 0, 1), torch.clamp(abd_outputs, 0, 1)
            liver_loss, abd_loss = criterion(liver_outputs, liver_masks), criterion(abd_outputs, abd_masks)
            total_loss = (liver_loss + abd_loss)/2
            total_loss.backward()
            optimizer.step()

            running_loss += total_loss.item()
            
        print(f'Fold [{fold+1}/{n_splits}], Epoch [{epoch+1}/{n_epochs}] - Loss: {running_loss / len(train_loader)}')
        
        scheduler.step(running_loss/len(train_loader))
        
        # Evaluation after each 5 epoch
        if (epoch + 1) % 10 == 0:
            model.eval()
            liver_dice_scores = []
            liver_hausdorff_distances = []
            liver_tprs = []
            liver_fprs = []

            abd_dice_scores = []
            abd_hausdorff_distances = []
            abd_tprs = []
            abd_fprs = []

            with torch.no_grad():
                for images, _, liver_masks, abd_masks in test_loader:
                    images, liver_masks, abd_masks = images.to(device), liver_masks.to(device), abd_masks.to(device)
                    liver_outputs, abd_outputs = model(images)
                    liver_outputs, abd_outputs = (liver_outputs > alpha).float().cpu().numpy(), (abd_outputs > alpha).float().cpu().numpy()     
                        
                    for i in range(liver_outputs.shape[0]):
                        liver_hausdorff_distances.append(hausdorff_distance(liver_masks[i].cpu().numpy(), liver_outputs[i]))
                        liver_dice_scores.append(dice_coef(liver_masks[i].cpu().numpy(), liver_outputs[i]))
                        liver_tprs.append(tpr(liver_masks[i].cpu().numpy(), liver_outputs[i]))
                        liver_fprs.append(fpr(liver_masks[i].cpu().numpy(), liver_outputs[i]))

                        abd_hausdorff_distances.append(hausdorff_distance(abd_masks[i].cpu().numpy(), abd_outputs[i]))
                        abd_dice_scores.append(dice_coef(abd_masks[i].cpu().numpy(), abd_outputs[i]))
                        abd_tprs.append(tpr(abd_masks[i].cpu().numpy(), abd_outputs[i]))
                        abd_fprs.append(fpr(abd_masks[i].cpu().numpy(), abd_outputs[i]))


            liver_avg_dice_coef, abd_avg_dice_coef = np.mean(liver_dice_scores), np.mean(abd_dice_scores)
            liver_avg_tpr, abd_avg_tpr = np.mean(liver_tprs), np.mean(abd_tprs)
            liver_avg_hausdorff, abd_avg_hausdorff = np.mean(liver_hausdorff_distances), np.mean(abd_hausdorff_distances)
            liver_avg_fpr, abd_avg_fpr = np.mean(liver_fprs), np.mean(abd_fprs)

            print(f'Fold [{fold+1}/{n_splits}] - Liver - Average Dice Coef: {liver_avg_dice_coef}, Average TPR: {liver_avg_tpr}, Average FPR: {liver_avg_fpr}, Average Hausdorff: {liver_avg_hausdorff}')
            print(f'Fold [{fold+1}/{n_splits}] - Abd Wall - Average Dice Coef: {abd_avg_dice_coef}, Average TPR: {abd_avg_tpr}, Average FPR: {abd_avg_fpr}, Average Hausdorff: {abd_avg_hausdorff}')

    model_save_path = os.path.join(model_dir, f'model_fold_{fold+1}.pth')
    torch.save(model.state_dict(), model_save_path)
    print(f'Model saved at {model_save_path}')

    # Evaluation
    model.eval()
    liver_dice_scores = []
    liver_hausdorff_distances = []
    liver_tprs = []
    liver_fprs = []

    abd_dice_scores = []
    abd_hausdorff_distances = []
    abd_tprs = []
    abd_fprs = []

    with torch.no_grad():
        for images, _, liver_masks, abd_masks in test_loader:
            images, liver_masks, abd_masks = images.to(device), liver_masks.to(device), abd_masks.to(device)
            liver_outputs, abd_outputs = model(images)
            liver_outputs, abd_outputs = (liver_outputs > alpha).float().cpu().numpy(), (abd_outputs > alpha).float().cpu().numpy()     
                   
            for i in range(liver_outputs.shape[0]):
                liver_hausdorff_distances.append(hausdorff_distance(liver_masks[i].cpu().numpy(), liver_outputs[i]))
                liver_dice_scores.append(dice_coef(liver_masks[i].cpu().numpy(), liver_outputs[i]))
                liver_tprs.append(tpr(liver_masks[i].cpu().numpy(), liver_outputs[i]))
                liver_fprs.append(fpr(liver_masks[i].cpu().numpy(), liver_outputs[i]))

                abd_hausdorff_distances.append(hausdorff_distance(abd_masks[i].cpu().numpy(), abd_outputs[i]))
                abd_dice_scores.append(dice_coef(abd_masks[i].cpu().numpy(), abd_outputs[i]))
                abd_tprs.append(tpr(abd_masks[i].cpu().numpy(), abd_outputs[i]))
                abd_fprs.append(fpr(abd_masks[i].cpu().numpy(), abd_outputs[i]))


    liver_avg_dice_coef, abd_avg_dice_coef = np.mean(liver_dice_scores), np.mean(abd_dice_scores)
    liver_avg_tpr, abd_avg_tpr = np.mean(liver_tprs), np.mean(abd_tprs)
    liver_avg_hausdorff, abd_avg_hausdorff = np.mean(liver_hausdorff_distances), np.mean(abd_hausdorff_distances)
    liver_avg_fpr, abd_avg_fpr = np.mean(liver_fprs), np.mean(abd_fprs)

    print(f'Fold [{fold+1}/{n_splits}] - Liver - Average Dice Coef: {liver_avg_dice_coef}, Average TPR: {liver_avg_tpr}, Average FPR: {liver_avg_fpr}, Average Hausdorff: {liver_avg_hausdorff}')
    print(f'Fold [{fold+1}/{n_splits}] - Abd Wall - Average Dice Coef: {abd_avg_dice_coef}, Average TPR: {abd_avg_tpr}, Average FPR: {abd_avg_fpr}, Average Hausdorff: {abd_avg_hausdorff}')
    f = open(performance_file, "a")
    print(f'Fold [{fold+1}/{n_splits}] - Liver - Average Dice Coef: {liver_avg_dice_coef}, Average TPR: {liver_avg_tpr}, Average FPR: {liver_avg_fpr}, Average Hausdorff: {liver_avg_hausdorff}', file=f)
    print(f'Fold [{fold+1}/{n_splits}] - Abd Wall - Average Dice Coef: {abd_avg_dice_coef}, Average TPR: {abd_avg_tpr}, Average FPR: {abd_avg_fpr}, Average Hausdorff: {abd_avg_hausdorff}', file=f)
    f.close()

Fold [1/3], Epoch [1/50] - Loss: 0.7212472690476311
Fold [1/3], Epoch [2/50] - Loss: 0.6518887645668454
Fold [1/3], Epoch [3/50] - Loss: 0.60649307568868
Fold [1/3], Epoch [4/50] - Loss: 0.5612568391693963
Fold [1/3], Epoch [5/50] - Loss: 0.5223543726735644
Fold [1/3], Epoch [6/50] - Loss: 0.4749639514419768
Fold [1/3], Epoch [7/50] - Loss: 0.4279542565345764
Fold [1/3], Epoch [8/50] - Loss: 0.38266949355602264
Fold [1/3], Epoch [9/50] - Loss: 0.3337985674540202
Fold [1/3], Epoch [10/50] - Loss: 0.2888990193605423
Fold [1/3] - Liver - Average Dice Coef: 7.899855518189725e-06, Average TPR: 0.0, Average FPR: 0.0, Average Hausdorff: inf
Fold [1/3] - Abd Wall - Average Dice Coef: 3.211664443369955e-05, Average TPR: 0.0, Average FPR: 0.0, Average Hausdorff: inf
Fold [1/3], Epoch [11/50] - Loss: 0.24549046986632878
Fold [1/3], Epoch [12/50] - Loss: 0.2087944812244839
Fold [1/3], Epoch [13/50] - Loss: 0.18056395981046888
Fold [1/3], Epoch [14/50] - Loss: 0.15554658406310612
Fold [1/3], Epoch 

## More Epochs

In [ ]:
train_loaders, test_loaders = load_data_loaders(os.path.join(dataloader_dir, 'train_loaders.pth'), os.path.join(dataloader_dir, 'test_loaders.pth'))
n_epochs = 50
additional_epochs = 50
alpha = 0.5

for fold in range(n_splits):
    train_loader = train_loaders[fold]
    test_loader = test_loaders[fold]

    efficientnet_model = get_efficientunet_b0_parallel(out_channels=1, concat_input=False, pretrained=False).to(device)
    optimizer = torch.optim.AdamW(efficientnet_model.parameters(), lr=1e-4, weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
    efficientnet_model.load_state_dict(torch.load(f"{os.getenv('BASE_DIR')}/Efficientunet/results/shared/models/model_fold_{fold+1}.pth"))
    # optimizer = torch.optim.Adam(model.parameters(), lr=1e-5, weight_decay=1e-4)
    criterion = dice_loss()

    # Continue Training
    for epoch in range(n_epochs, n_epochs + additional_epochs):
        efficientnet_model.train()
        running_loss = 0.0
        for images, _, liver_masks, abd_masks in train_loader:
            images, liver_masks, abd_masks = images.to(device), liver_masks.to(device), abd_masks.to(device)

            optimizer.zero_grad()
            liver_outputs, abd_outputs = efficientnet_model(images)
            liver_outputs, abd_outputs = torch.clamp(liver_outputs, 0, 1), torch.clamp(abd_outputs, 0, 1)
            liver_loss, abd_loss = criterion(liver_outputs, liver_masks), criterion(abd_outputs, abd_masks)
            total_loss = (liver_loss + abd_loss)/2
            total_loss.backward()
            optimizer.step()

            running_loss += total_loss.item()
            
        print(f'Fold [{fold+1}/{n_splits}], Epoch [{epoch+1}/{n_epochs}] - Loss: {running_loss / len(train_loader)}')
        
        scheduler.step(running_loss/len(train_loader))
        
        # Evaluation after each 10 epoch
        if (epoch + 1) % 10 == 0:
            efficientnet_model.eval()
            liver_dice_scores = []
            liver_hausdorff_distances = []
            liver_tprs = []
            liver_fprs = []

            abd_dice_scores = []
            abd_hausdorff_distances = []
            abd_tprs = []
            abd_fprs = []

            with torch.no_grad():
                for images, _, liver_masks, abd_masks in test_loader:
                    images, liver_masks, abd_masks = images.to(device), liver_masks.to(device), abd_masks.to(device)
                    liver_outputs, abd_outputs = efficientnet_model(images)
                    liver_outputs, abd_outputs = (liver_outputs > alpha).float().cpu().numpy(), (abd_outputs > alpha).float().cpu().numpy()     
                        
                    for i in range(liver_outputs.shape[0]):
                        liver_hausdorff_distances.append(hausdorff_distance(liver_masks[i].cpu().numpy(), liver_outputs[i]))
                        liver_dice_scores.append(dice_coef(liver_masks[i].cpu().numpy(), liver_outputs[i]))
                        liver_tprs.append(tpr(liver_masks[i].cpu().numpy(), liver_outputs[i]))
                        liver_fprs.append(fpr(liver_masks[i].cpu().numpy(), liver_outputs[i]))

                        abd_hausdorff_distances.append(hausdorff_distance(abd_masks[i].cpu().numpy(), abd_outputs[i]))
                        abd_dice_scores.append(dice_coef(abd_masks[i].cpu().numpy(), abd_outputs[i]))
                        abd_tprs.append(tpr(abd_masks[i].cpu().numpy(), abd_outputs[i]))
                        abd_fprs.append(fpr(abd_masks[i].cpu().numpy(), abd_outputs[i]))


            liver_avg_dice_coef, abd_avg_dice_coef = np.mean(liver_dice_scores), np.mean(liver_dice_scores)
            liver_avg_tpr, abd_avg_tpr = np.mean(liver_tprs), np.mean(abd_tprs)
            liver_avg_hausdorff, abd_avg_hausdorff = np.mean(liver_hausdorff_distances), np.mean(abd_hausdorff_distances)
            liver_avg_fpr, abd_avg_fpr = np.mean(liver_fprs), np.mean(abd_fprs)

            print(f'Fold [{fold+1}/{n_splits}] - Average Dice Coef: {liver_avg_dice_coef}, Average TPR: {liver_avg_tpr}, Average FPR: {liver_avg_fpr}, Average Hausdorff: {liver_avg_hausdorff}')
            print(f'Fold [{fold+1}/{n_splits}] - Average Dice Coef: {abd_avg_dice_coef}, Average TPR: {abd_avg_tpr}, Average FPR: {abd_avg_fpr}, Average Hausdorff: {abd_avg_hausdorff}')

    model_save_path = os.path.join(model_dir, f'model_fold_{fold+1}_continued.pth')
    torch.save(efficientnet_model.state_dict(), model_save_path)
    print(f'Model saved at {model_save_path}')

    # Evaluation
    efficientnet_model.eval()
    liver_dice_scores = []
    liver_hausdorff_distances = []
    liver_tprs = []
    liver_fprs = []

    abd_dice_scores = []
    abd_hausdorff_distances = []
    abd_tprs = []
    abd_fprs = []

    with torch.no_grad():
        for images, _, liver_masks, abd_masks in test_loader:
            images, liver_masks, abd_masks = images.to(device), liver_masks.to(device), abd_masks.to(device)
            liver_outputs, abd_outputs = efficientnet_model(images)
            liver_outputs, abd_outputs = (liver_outputs > alpha).float().cpu().numpy(), (abd_outputs > alpha).float().cpu().numpy()     
                   
            for i in range(liver_outputs.shape[0]):
                liver_hausdorff_distances.append(hausdorff_distance(liver_masks[i].cpu().numpy(), liver_outputs[i]))
                liver_dice_scores.append(dice_coef(liver_masks[i].cpu().numpy(), liver_outputs[i]))
                liver_tprs.append(tpr(liver_masks[i].cpu().numpy(), liver_outputs[i]))
                liver_fprs.append(fpr(liver_masks[i].cpu().numpy(), liver_outputs[i]))

                abd_hausdorff_distances.append(hausdorff_distance(abd_masks[i].cpu().numpy(), abd_outputs[i]))
                abd_dice_scores.append(dice_coef(abd_masks[i].cpu().numpy(), abd_outputs[i]))
                abd_tprs.append(tpr(abd_masks[i].cpu().numpy(), abd_outputs[i]))
                abd_fprs.append(fpr(abd_masks[i].cpu().numpy(), abd_outputs[i]))


    liver_avg_dice_coef, abd_avg_dice_coef = np.mean(liver_dice_scores), np.mean(liver_dice_scores)
    liver_avg_tpr, abd_avg_tpr = np.mean(liver_tprs), np.mean(abd_tprs)
    liver_avg_hausdorff, abd_avg_hausdorff = np.mean(liver_hausdorff_distances), np.mean(abd_hausdorff_distances)
    liver_avg_fpr, abd_avg_fpr = np.mean(liver_fprs), np.mean(abd_fprs)

    print(f'Fold [{fold+1}/{n_splits}] - Average Dice Coef: {liver_avg_dice_coef}, Average TPR: {liver_avg_tpr}, Average FPR: {liver_avg_fpr}, Average Hausdorff: {liver_avg_hausdorff}')
    print(f'Fold [{fold+1}/{n_splits}] - Average Dice Coef: {abd_avg_dice_coef}, Average TPR: {abd_avg_tpr}, Average FPR: {abd_avg_fpr}, Average Hausdorff: {abd_avg_hausdorff}')
    f = open(performance_file, "a")
    print(f'Fold [{fold+1}/{n_splits}] - Average Dice Coef: {liver_avg_dice_coef}, Average TPR: {liver_avg_tpr}, Average FPR: {liver_avg_fpr}, Average Hausdorff: {liver_avg_hausdorff}', file=f)
    print(f'Fold [{fold+1}/{n_splits}] - Average Dice Coef: {abd_avg_dice_coef}, Average TPR: {abd_avg_tpr}, Average FPR: {abd_avg_fpr}, Average Hausdorff: {abd_avg_hausdorff}', file=f)
    f.close()

C:\Users\User\AppData\Local\Temp\ipykernel_24192\2090685341.py:37: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_loaders = torch.load(train_path)
C:\Users\User\AppData

Fold [1/3], Epoch [51/50] - Loss: 0.09428880280918545
Fold [1/3], Epoch [52/50] - Loss: 0.08763845264911652
Fold [1/3], Epoch [53/50] - Loss: 0.08453312516212463
Fold [1/3], Epoch [54/50] - Loss: 0.08159866432348888
Fold [1/3], Epoch [55/50] - Loss: 0.07795201738675435
Fold [1/3], Epoch [56/50] - Loss: 0.07519110871685876
Fold [1/3], Epoch [57/50] - Loss: 0.07356542845567067
Fold [1/3], Epoch [58/50] - Loss: 0.06916660401556227
Fold [1/3], Epoch [59/50] - Loss: 0.06708403262827131
Fold [1/3], Epoch [60/50] - Loss: 0.065254012743632
Fold [1/3] - Average Dice Coef: 0.8018028140068054, Average TPR: 0.970007598400116, Average FPR: 0.06243303790688515, Average Hausdorff: 253.0085306985836
Fold [1/3] - Average Dice Coef: 0.8018028140068054, Average TPR: 0.018469158560037613, Average FPR: 5.421667026439536e-08, Average Hausdorff: inf
Fold [1/3], Epoch [61/50] - Loss: 0.06390999423133002
Fold [1/3], Epoch [62/50] - Loss: 0.061541070540746055
Fold [1/3], Epoch [63/50] - Loss: 0.0591247263881895

In [ ]:
import time
import matplotlib.pyplot as plt

efficientnet_model = get_efficientunet_b0_parallel(out_channels=1, concat_input=False, pretrained=False).to(device)
efficientnet_model.load_state_dict(torch.load('./model_fold_1.pth'))
efficientnet_model.eval()

dice_scores = []
hausdorff_distances = []
tprs = []
fprs = []
time_taken = []

with torch.no_grad():
    for j, (images, _, liver_masks, abd_masks) in enumerate(test_loader):
        if j >= 5:
            break
        images, liver_masks, abd_masks = images.to(device), liver_masks.to(device), abd_masks.to(device)
        start_time = time.time()
        liver_outputs, abd_outputs = efficientnet_model(images)
        liver_outputs, abd_outputs = (liver_outputs > alpha).float().cpu().numpy(), (abd_outputs > alpha).float().cpu().numpy()
        end_time = time.time()
        time_taken.append(end_time - start_time)
        print(f"Prediction time for batch {j} with batch size {batch_size}: {end_time - start_time} seconds")
        
        for i in range(liver_outputs.shape[0]):
            # dice_scores.append(dice_coef(masks[i].cpu().numpy(), outputs[i]))
            # tprs.append(tpr(masks[i].cpu().numpy(), outputs[i]))
            # fprs.append(fpr(masks[i].cpu().numpy(), outputs[i]))
            # hausdorff_distances.append(hausdorff_distance(masks[i].cpu().numpy(), outputs[i]))
            
            liver_mask = liver_masks[i].cpu().numpy().transpose(1, 2, 0)
            abd_mask = abd_masks[i].cpu().numpy().transpose(1, 2, 0)
            image = images[i].cpu().numpy().transpose(1, 2, 0)
            liver_prediction = liver_outputs[i].transpose(1, 2, 0)
            abd_prediction = abd_outputs[i].transpose(1, 2, 0)
            
            plt.figure(figsize=(16, 8))
            plt.subplot(241)
            plt.title('Image')
            plt.imshow(image[:,:,0], cmap='gray')
            plt.subplot(242)
            plt.title('Liver Label')
            plt.imshow(liver_mask[:,:,0], cmap='gray')
            plt.subplot(243)
            plt.title('Prediction on Liver')
            plt.imshow(liver_prediction, cmap='gray')
            plt.subplot(244)
            plt.title("Overlayed Images")
            plt.imshow(liver_mask.astype(float) / np.max(mask), cmap='gray')
            plt.imshow(liver_prediction, cmap='jet', alpha=0.5)
            plt.subplot(245)
            plt.title('Image')
            plt.imshow(image[:,:,0], cmap='gray')
            plt.subplot(246)
            plt.title('Abd Label')
            plt.imshow(abd_mask[:,:,0], cmap='gray')
            plt.subplot(247)
            plt.title('Prediction on Abd')
            plt.imshow(abd_prediction, cmap='gray')
            plt.subplot(248)
            plt.title("Overlayed Images")
            plt.imshow(abd_mask.astype(float) / np.max(mask), cmap='gray')
            plt.imshow(abd_prediction, cmap='jet', alpha=0.5)
            
            
            plt.savefig(os.getenv('BASE_DIR') + f'/Efficientunet/overlay/fold1_{j}_{i}.png')
            plt.close()
            
    
average_dice_coef = np.mean(dice_scores)
average_tpr = np.mean(tprs)
average_fpr = np.mean(fprs)
average_hausdorff = np.mean(hausdorff_distances)
average_time_taken = np.mean(time_taken)/batch_size

print(f'Average Dice Coef: {average_dice_coef}, Average TPR: {average_tpr}, Average FPR: {average_fpr}, Average Hausdorff Distance: {average_hausdorff}, Average Prediction Time: {average_time_taken}')

C:\Users\Jeff\AppData\Local\Temp\ipykernel_50580\3074383525.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('./model_fold_1.pth'))


Prediction time for batch 0 with batch size 4: 0.09335112571716309 seconds
Prediction time for batch 1 with batch size 4: 0.08713650703430176 seconds
Prediction time for batch 2 with batch size 4: 0.11528801918029785 seconds
Prediction time for batch 3 with batch size 4: 0.10626649856567383 seconds
Prediction time for batch 4 with batch size 4: 0.09590435028076172 seconds
Average Dice Coef: nan, Average TPR: nan, Average FPR: nan, Average Hausdorff Distance: nan, Average Prediction Time: 0.02489732503890991


c:\Users\Jeff\anaconda3\envs\ultrasound\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\Jeff\anaconda3\envs\ultrasound\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
